## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np
# Import API key
from config import g_key
from config import weather_api_key

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv ("Weather_Database/WeatherPy_Database.csv")

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,2021-03-28 01:40:48,22.0752,-159.3190,78.80,73,20,14.97,few clouds
9,9,Vaini,TO,2021-03-28 01:38:19,-21.2000,-175.2000,86.00,74,40,9.22,light rain
15,15,La Libertad,EC,2021-03-28 01:40:52,-2.2333,-80.9000,75.13,81,44,5.19,scattered clouds
16,16,Kismayo,SO,2021-03-28 01:40:52,-0.3582,42.5454,78.94,78,73,10.07,broken clouds
18,18,Karratha,AU,2021-03-28 01:40:53,-20.7377,116.8463,89.13,47,42,2.42,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                223
City                   223
Country                223
Date                   223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,78.80,few clouds,22.0752,-159.3190,
9,Vaini,TO,86.00,light rain,-21.2000,-175.2000,
15,La Libertad,EC,75.13,scattered clouds,-2.2333,-80.9000,
16,Kismayo,SO,78.94,broken clouds,-0.3582,42.5454,
18,Karratha,AU,89.13,scattered clouds,-20.7377,116.8463,
20,Butaritari,KI,80.65,moderate rain,3.0707,172.7902,
21,Alofi,NU,84.20,light rain,-19.0595,-169.9187,
24,Japura,ID,83.44,overcast clouds,-0.3167,102.3500,
28,Jacqueville,CI,80.60,clear sky,5.2052,-4.4146,
31,Atuona,PF,81.21,overcast clouds,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
#Recall that when trying to parse the data from an API request, we need to use a try-except block to handle the error with 
#a statement and continue the retrieval.

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,78.80,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
9,Vaini,TO,86.00,light rain,-21.2000,-175.2000,Keleti Beach Resort
15,La Libertad,EC,75.13,scattered clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique
16,Kismayo,SO,78.94,broken clouds,-0.3582,42.5454,Agan-Hotel
18,Karratha,AU,89.13,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
20,Butaritari,KI,80.65,moderate rain,3.0707,172.7902,Isles Sunset Lodge
21,Alofi,NU,84.20,light rain,-19.0595,-169.9187,Taloa Heights
24,Japura,ID,83.44,overcast clouds,-0.3167,102.3500,Rumah mbah Jamah
28,Jacqueville,CI,80.60,clear sky,5.2052,-4.4146,Hotel Le Martin Pecheur
31,Atuona,PF,81.21,overcast clouds,-9.8000,-139.0333,Villa Enata


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,78.80,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
9,Vaini,TO,86.00,light rain,-21.2000,-175.2000,Keleti Beach Resort
15,La Libertad,EC,75.13,scattered clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique
16,Kismayo,SO,78.94,broken clouds,-0.3582,42.5454,Agan-Hotel
18,Karratha,AU,89.13,scattered clouds,-20.7377,116.8463,ibis Styles Karratha


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))